In [1]:
import torch
import argparse
import numpy as np
import torch.nn as nn
from config import Config
from torch.nn import functional as F
from utils.tokenizer import Tokenizer, load_vocab
from model.pretrain_model import ToEmbedding
from torch.utils.data import Dataset, DataLoader

In [2]:
class VocabDataset(Dataset):
    def __init__(self, filename, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length
        with open(filename, 'r+') as f:
            texts = f.readlines()
        self.data = self.process_data(texts)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]

    def process_data(self, texts):
        data = []
        for text in texts:
            tokens = tokenizer.tokenize(text)
            if len(tokens) > self.max_length - 2:
                tokens = tokens[: (self.max_length - 2)]
            tokens_id, tokens_masks, segment_id = self.tokenizer.create_feature(tokens)
            data.append((tokens_id, tokens_masks, segment_id, text.strip()))
        return data

def collate_fn(batch_data):
    batch_size = len(batch_data)
    batch_data = list(zip(*batch_data))
    lengths = [len(x) for x in batch_data[0]]
    max_len = max(lengths)
    tokens_id = torch.LongTensor(batch_size, max_len).fill_(0)
    tokens_masks = torch.LongTensor(batch_size, max_len).fill_(0)
    segment_id = torch.LongTensor(batch_size, max_len).fill_(0)
    
    for i in range(batch_size):
            tokens_id[i, :len(batch_data[0][i])] = torch.LongTensor(batch_data[0][i])
            tokens_masks[i, :len(batch_data[1][i])] = torch.LongTensor(batch_data[1][i])
            segment_id[i, :len(batch_data[2][i])] = torch.LongTensor(batch_data[2][i])
    return (tokens_id, tokens_masks, segment_id, batch_data[3])

def dataloader(filename, batch_size, tokenizer, max_seq_length=200,
                    dropout=0.01, shuffle=False, lower=True, num_workers=8,
                    collate_fn=collate_fn):
    dataset = VocabDataset(filename, tokenizer, max_seq_length)
    data_loader = DataLoader(dataset=dataset, 
                            batch_size=batch_size, 
                            shuffle=shuffle, 
                            pin_memory=True, 
                            num_workers=num_workers, 
                            collate_fn=collate_fn)
    return data_loader

In [3]:
vocab = load_vocab('dataset/vocab.txt')
tokenizer = Tokenizer(vocab)

data_loader = dataloader('dataset/to_embedding/vocab.txt', 32, tokenizer)

/home/su/miniconda3/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
cfg = Config()
cfg.load_config('config.json')

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
checkpoint = torch.load('saved_models/biobert-base-cased-v1.1.pth')

model = ToEmbedding(embedding_dim=100, vocab_size=len(vocab), device=device, checkpoint=checkpoint,**cfg.config)
model.to(device)
dense_layer = torch.load('saved_models/to_embedding.pt')

model.dense_layer.load_state_dict(dense_layer['state_dict'][0])

Config loaded from file config.json


<All keys matched successfully>

In [9]:
pooling = 'mean'
with torch.no_grad():
    res = []
    for idx, (inputs, masks, token_type_ids, metadata) in enumerate(data_loader):
        inputs = inputs.to(device)
        masks = masks.to(device)
        token_type_ids = token_type_ids.to(device)
        
        encoder_output = model.bert(inputs, masks, token_type_ids)[0]
        encoder_output = model.dense_layer(encoder_output).detach()
        
        embeddings = []
        
        if pooling == 'first':
            embeddings = encoder_output[:, 0, :]
        elif pooling == 'sum' or pooling == 'mean':
            masks = masks.detach()
            masks = torch.nn.functional.pad(masks[:,2:],(1,1))
            masks = masks.unsqueeze(-1).expand(encoder_output.shape)
            sub_embeddings = (masks.to(torch.float) * encoder_output)
            embeddings = sub_embeddings.sum(dim=1)
            
            if pooling == 'mean':
                masks = masks[:,:,0].sum(dim=-1).unsqueeze(1)
                embeddings = embeddings / masks.to(torch.float)
            
        for embedding, each_metadata in zip(embeddings,metadata):
            res.append((each_metadata, embedding.tolist()))

In [13]:
len(res)

103223

In [15]:
for item in res:
    print(item[0])

,
and
of
the
.
in
)
(
cancer
to
a
with
expression
that
/
is
cells
cell
for
was
as
breast
by
we
tumor
:
were
gene
patients
lung
from
or
human
tissues
genes
metastasis
on
this
an
study
tissue
protein
are
%
which
prostate
be
normal
tumors
carcinoma
associated
gastric
results
growth
role
levels
1
these
cancers
disease
significantly
our
may
0
invasion
its
compared
liver
increased
lines
including
have
p53
proliferation
2
between
found
treatment
using
has
progression
ovarian
not
analysis
=
target
but
also
been
potential
p
colorectal
at
stage
pathway
samples
most
primary
+
signaling
factor
showed
survival
bladder
through
mice
both
clinical
pancreatic
colon
high
receptor
data
identified
development
mrna
than
thyroid
3
migration
apoptosis
hcc
such
novel
renal
other
therapeutic
expressed
suppressor
can
activity
well
higher
methods
used
it
their
activation
model
;
oncogenic
risk
mutations
common
metastatic
epithelial
overexpression
crc
cases
present
brain
skin
here
cervical
studies
level
effects
d

runx1
2007
elisa
describes
consisted
offers
his
untranslated
neutrophils
nasal
killer
pathophysiological
kdm2a
muscle-invasive
concept
attenuate
minimally
pan-cancer
incident
na
4t1
atf4
rest
allows
38-69
altering
a3b
cxcr2
cell-specific
tuberous
regimens
microbial
c-kit
overview
proteomics
il-10
platinum
scaffold
shp2
native
mir-19a
fap
round
dementia
depends
co-expressed
2d
v2
disrupted
until
h3k4
mds
variability
determinant
benefits
oa
feasible
make
p53-null
adaptive
2000
issue
wilms'
p16ink4a
vdac1
phosphatidylinositol
ctcl
glaucoma
sw620
final
verify
give
opportunities
reciprocal
slit2
concerning
nc
fed
reductase
sepsis
mcc
stk11
establishing
pronounced
question
select
mir-506
pthrp
j
chamber
ppi
movement
biosynthesis
adolescents
angiomyolipomas
dependence
ctla-4
changed
codon
communication
ia
activators
gains
downregulates
space
routine
pluripotency
d2
beclin1
all-cause
ap-1
antiproliferative
malt
schwannomas
pubmed
functioning
pecam1
neuroectodermal
morphologically
mmp-7
augment

tbp
rtt
death-1
wherein
refers
ai
surfactant
schizophrenia
drug-resistance
gof
ap4
trap
trpm2
nlrp3
pneumonitis
feeding
orthotopically
hydrogen
infiltrates
urethra
forebrain
ang
400
fumarate
completion
1999
fusion-positive
examines
morphogenetic
sustain
contraction
mir-98
ancestry
ing1
grb2
necrotizing
pdgfb
catalyzes
mir-30a-5p
mir-196a
aldh1a3
pharmacologically
epithelial-derived
chronically
antagonizes
carries
phosphorylates
wrn
disc
high-density
disease-associated
cg
scheduled
socs2
mir-613
hoxb13
beam
prion
ceacam1
exclude
compensatory
ipca
estrogens
cytometric
psmb5
nrf2-mediated
adenoviruses
tfe3
sirt7
tip
cervicitis
lrp
adhesions
lacks
proneural
nedd4
5-htt
keratitis
resorption
kim-1
tlr3
fat1
tk
yang
confirms
swelling
icr
zfx
non-tumour
plasmacytoma
u19
plaques
microenvironmental
ect2
y-box
utrs
analogs
sigmoid
ulk1
cyclooxygenase
octamer-binding
aggregates
zones
autoantibody
hras-transformed
krasc118s
pim-3
cerebrum
rnaseq
flt3
proportions
ic
northern
incorporated
axial
flank

removing
theranostic
african-american
flexible
palpable
instable
neuroblastic
chi-square
plotter
trib2
histotype
trb3
hinders
get
hpa
sexually
-5p
synaptophysin
sham
boundaries
trx1
colony-stimulating
ladc
eya4
ppis
satisfactory
mycobacterium
siah2
f344
chfr
peg
txn
4a
187
chc
204
prognoscan
hrh1
tyro3
irs2
lpa1
phgdh
rap80
tsp1
ultrasonographic
mir-149
endometriotic
epidemiol
proc
natl
acad
disappeared
scd
fbp1
principles
casein
microrna-383
t-lbl
exciting
shifting
gst
steady
adapted
mir-101-3p
visualize
ema
client
pools
deformity
sessile
coprevalent
publication
primates
lesional
nursing
canadian
carcinogenicity
connecting
apoptosis-associated
sqc
igf2-low
granulomatosis
infertility
mir-15a
colons
nephrology
tremendous
record
hsa-mir-16
cenpe
sema3f
reprogrammed
polyadenylation
rnase
condensation
mta2
round-cell
a7
admixed
sheep
approval
au-rich
mir-140-5p
statin
mpc1
pruritic
nifedipine
debilitating
sncg
ovca
workup
ms-k
151
insensitive
cpt1a
tilt
sputum
favoring
327
hits
192
adverse

s6k2
forty-two
demyelination
prkd1
phospho-p38
uncoupling
tomato
lmna
manufacturing
tmem174
moles
5-aza-2-deoxycytidine
tim-4
zd55-shmycn
prompt
arhgef7
hex
pmdc05
lps-pdc
tcl1a
pursue
rps20
regorafenib
cyclinb1
undescribed
apc-deficient
ari
sox18
vasoconstriction
hypoxia-driven
deep-sequencing
biomolecular
ire1
myoblasts
thorough
hey2
selp
usp8
nos1
mnk1
calu-1
ppp1r11
lymphoepithelial
estrogen-independent
ppp2r2a
psen1
moloney
lowers
spliceosome
customized
fifty-four
adenosarcoma
endovascular
krt14
trib1
lysosome
radiolabeled
deguelin
sequestosome-1
medicinal
polyendocrinopathy
werner
move
ipsc
atrogin-1
deubiquitination
equivalents
vip
fk506
roughly
epidermolysis
chka
antitumoral
sea
spc-induced
isthmus
flcn-deficient
sucrose
high-sensitivity
bilirubin
concentrate
cartilaginous
girl
come
drug-sensitive
cgp
tmb
neo-angiogenesis
subcomplex
transplantable
proline
65-73
187-197
226-234
74-82
ras-mapk
agenesis
down-modulation
lrh
cell-of-origin
chl1
-off-flavor
pro-migratory
u133
interpr

...

-jiang
bst
atp2b1
anthracycline-dependent
loss-of-fat1-function
transgene-induced
uke-1
inter-malignant
zfx-mediated
pca-lda
prodifferentiation
rs4246444
vav3-sirna-transfected
three-membered
antigen-mediated
icgn
megn
ip1--the
comminuted
814
tyr1349
moloney-murine
tnfrsf8
tnfsf8
mir-101-dnmt3a
aqp5-sirna
gasotransmitters
ho-2
pkn2
telo2
mrpl9
mttl1
klhl20
pi4kb
th1-oriented
nanovectors
c-dim12
nurr1-regulated
p-ezrin
rem2a
cytohesin
nett
tp53-defective
hbsag-seropositive
hbsag-seronegative
milliliter
casp8-ir
polymer-nanoclay-based
validations-in
biopsies-we
ahscs
cleaned
suppressor-regulating
set8-
cpth2
neutrophil-derived
lung-protective
pneumonia-inducing
scribbled
d2-mediated
firefighting
paclitaxel-mediated
cyp3a11
tlr4-independent
iemg
nci-h716
pgrmc1-transfected
sex-mismatched
andrunx1
nci-h82
domain-function
tyr225cys
674a
y225c
tmd8
bap1-mutant
anoestrus
spayed
wax-embedded
cell-cancer-associated
residualized
bitragion
lip-chin
utc
w4p
lhb
ptp4a3-driven
ptp4a3-mediated
straig

epp62
epp65
epp5
epp71
epp74
hoxb13-deficient
hoxb13-positive
melignant
gstt1-deleting
sac-2
1987-1989
cspc
extraribosomal
non-smallcell
oncomirna
klf6-e2f1
reversal-adrenal
ahc
reg-i
pd1-pdl1-targeted
ym201636
15mb
4-thiazolidinone
les-2194
les-3377
les-3640
4-thiazolidinones
progression-opposing
6n
nephron-based
erbb4-mapk
microrna-130b-3p
fhl124
nephrotoxin-induced
cfe-mri
reactants
excretory-secretory
pup
gna13-3
-utr-reporter
cell-growth-related
0003221
kaplan-meir
hpa-positive
tc-rpmm
memberships
mycn-mir-26a-5p-lin28b
diurnally
lxr-dependent
4-dihydropyridine
vdie-2n
7-stimulated
3-p21
haphazardly
regulators--histone
erastin
crocodile
cdk-1
perinasal
6q15-deletions
barghout
zepeda
vincent
azad
steed
postovit
rrgs
keratorefractive
anti-oncostatin
preparatory
subjectvies
wsd
igf1-pi3k-dependent
enzyme--phosphoglycerate
sord
cdv3
elas-creer
brl
epo-compounds
bf-rtk
uantitative
intelectin
dcm-ds-treated
nbnma-induced
compulsory
cullin-ring-containing
19year
alk-tkis
mir-642b-3p
pro-

preauricular
act-associated
arosarena
dela
cadena
denny
bryant
thorpe
safadi
ofcs
target-snps
browsed
betatrophin
pb-arv567es
showcase
dropping
medium-recurrence
achn-bo5
pressure-time
integrals
in-shoe
primary-resected
sdm
viridis
side-based
field-based
interferon-alpha2
3loxp
gusb-null
ep4-knockdown
kruscal-wallis-h-test
mismatch-specific
adenine-
thymine-dna
mismatched
hdac1-mediated
non-clinical
89zr-dfo-amg102
-styrylphenoxy
butanoyl
-l-4-hydroxyprolyl
-thiazolidine
benzyloxycarbonyl-thioprolyl-thioprolinal
qatar
ta-rcc
trial-level
-79tt
poor-prognostic
gw1100
vadt
cbna
hmda-7
vwa2
masculinization
sw13-
dna-lna
turban
acylaminoacyl-peptidase
dystroglycan
dag1
k27m-mutant
gata2-
nfsa
pichia
pastoris
n-status
t-status
lc3-positive
invadopodia-related
il20rb
r213x
kayser-fleischer
shr-socs1-transduced
metallo-proteinases
cd-10
nkg2d-l
cbrs
sbrs
baroreceptor
ud-scc2
upci-scc72
002059
0000190
circ-sfmbt2
104075
100338
0136666
0000523
0006427
0089105
circagfg1
circepsti1
circtfrc
circ-d

assuring
chn2
ist
duct-to-duct
forget
rheb1
wccn
1177
mbic
deprives
co-twin
attention-control
non-cardiovascular
gyncsm
risk-adjusted
fbxo31-induced
electrophoresis-time
flight-mass
chordata-specific
ape1-knockout
oligos
uca1-overexpressing
piloted
fender
peripheries
aortogram
sensory-specific
a-c1
hras-like
egfrt790
klf17-mediated
klotho-treated
mo3
epigastralgia
t-antigens
bio-active
splice-variant
rdeb-associated
prothoracic
prp8
cyp307a2
spookier
spok
ecdysone-biosynthetic
hpv-immortalized
18f-fdg-pet
d299g
t399i
darwinian-type
wsc
pymt-
wnt1-driven
qg
rs34330
morc1
morc4
g-lisa-based
immunostaining-based
lec-mediated
tcm-treated
-cacgtg-3
advisory
adjudged
nonappearance
fedex
rs-hepcko
rvecs
nsr
acth-immunopositive
monocryl
4-0
years'
nmd-induced
ad2
asc-stimulated
carcinoma-asc
mek-targeting
ohsu
predesignated
mrx34
liposome-formulated
lmp1-stimulated
cribriform-morular
morular
ceratioid
leontiasis
ossea
jmjd2a-regulated
nonproductively
cyld-defective
npm1-luciferase
ei950
histoc

geosmin-producing
geosmin
off-flavors
pcdc4
icr4
pttg1-overexpressing
histocompatibility-mismatched
vcta
deiodination
decarboxylation
mb-pdt
bone-liver-thymus
hu-blt
hu-at
77-gene
regulable
alkomas
18-gauge
calix
pdpe
thermogelling
situ-forming
mushrooms
ergosterol
6-oap
ser611
ser613
arg609
interleukin-6-induced
super-resolution
incontinentia
pigmenti
rs8154
meqtl
genotype-tissue
lymphedemas
adnexes
45-63
refuge
virchow's
radio-resistant
mir-omic
microhemorrhage
macrohemorrhage
linear-by-linear
atm-null
pre-ri
lst1
leucocyte-specific
bbe8
stp1
lsu2
dc3000
recirculate
233-gene
based-comparative
tmucih
nrf2-ho-1
ip5
heuristic
hbi
tomosynthesis
filtered-back
fpb
needleless
dripping
lidocaine
mesangiopathy
acid-inducing
re-analyze
st18-depleted
-omics
akt-phosphorylation
kettering-integrated
cen7
effector-loop
procainamide
maraviroc
mir-218-tpd52
ogx-011
findings--endometrial
chi2
00005
tweets
sentiment
users'
tobreast
xanthoma
verrucopapillary
sclerosants
thermoplasty
tumor-to-nontumor
t

In [22]:
embs = [ i[1] for i in res]

In [23]:
pad = np.zeros(100).tolist()
unk = np.random.rand(100).tolist()
embs.append(pad)
embs.append(unk)


In [26]:
embs = np.array(embs)

In [27]:
np.save('embeddings.npy', embs)

In [1]:
import torch
import argparse
import numpy as np
import torch.nn as nn
from config import Config, Logger
from torch.nn import functional as F
from utils.tokenizer import Tokenizer, load_vocab

In [2]:
from data.dataset import ReBagDataLoader

In [3]:
logger = Logger()

In [4]:
vocab = load_vocab('dataset/vocab.txt')
tokenizer = Tokenizer(vocab)

In [5]:
data_loader = ReBagDataLoader('dataset/re/test.txt', 160, 0,logger, tokenizer)

2021-05-16 11:58:33,935 [DEBUG]: -----------read data-----------
2021-05-16 11:58:34,503 [DEBUG]: dataset/re/test.txt has data 78956
/home/su/miniconda3/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
for i in data_loader:
    print(i)

['in', 'this', 'single', '-', 'center', 'retrospective', 'study', ',', '83', 'consecutive', 'patients', 'with', 'o', '##sse', '##ous', 'meta', '##sta', '##si', '##zed', 'pro', '##state', 'cancer', 'were', 'evaluated', ',', 'who', 'had', 'primarily', 'been', 'treated', 'by', 'sub', '##cap', '##sul', '##ar', 'bilateral', 'or', '##chi', '##ec', '##tom', '##y', '.']
['background', ':', 'ma', '##li', '##gnant', 'p', '##le', '##ural', 'me', '##so', '##the', '##lio', '##ma', '(', 'm', '##pm', ')', 'is', 'an', 'aggressive', 'neo', '##p', '##las', '##m', 'arising', 'from', 'me', '##so', '##the', '##lial', 'lining', 'of', 'p', '##le', '##ura', '.']['in', 'the', 'present', 'study', ',', 'the', 'anti', '-', 'tumor', 'action', 'of', 'me', '##sen', '##chy', '##mal', 'stem', 'cells', '(', 'm', '##s', '##cs', ')', 'has', 'been', 'examined', 'in', 'a', 'mouse', 'model', 'of', 'breast', 'cancer', 'with', 'emphasize', 'on', 'tumor', 'growth', ',', 'an', '##gio', '##genesis', 'and', 'c', '-', 'my', '##c',

tokens ['this', 'study', 'highlights', 'the', 'significant', 'difference', 'in', 'l', '##1', 'l', '##1', '-', 'or', '##f', '##2', 'expression', 'between', 'c', '##t', '##cs', 'and', 'normal', 'samples', '.'], tokens_id [101, 1142, 2025, 12976, 1103, 2418, 3719, 1107, 181, 1475, 181, 1475, 118, 1137, 2087, 1477, 2838, 1206, 172, 1204, 6063, 1105, 2999, 8025, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], pos1 is [0, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

tokens ['e', '##uka', '##ryo', '##tic', 'el', '##ong', '##ation', 'factor', '2', 'kinase', '(', 'e', '##ef', '##2', '##k', ')', ',', 'an', 'emerging', 'molecular', 'target', 'for', 'cancer', 'therapy', ',', 'contributes', 'to', 'cancer', 'proliferation', ',', 'cell', 'survival', ',', 'tumor', '##ige', '##nes', '##is', ',', 'and', 'invasion', ',', 'disease', 'progression', 'and', 'drug', 'resistance', '.'], tokens_id [101, 174, 12658, 26503, 2941, 8468, 4553, 1891, 5318, 123, 24779, 113, 174, 11470, 1477, 1377, 114, 117, 1126, 8999, 9546, 4010, 1111, 4182, 7606, 117, 17705, 1106, 4182, 23766, 117, 2765, 8115, 117, 14601, 13417, 3965, 1548, 117, 1105, 4923, 117, 3653, 16147, 1105, 3850, 4789, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], pos1 is [0, 101, 102, 103, 104, 105, 106, 107, 108, 

tokens ['p', '##le', '##ural', 'ma', '##li', '##gnant', 'me', '##so', '##the', '##lio', '##ma', '(', 'm', '##pm', ')', 'is', 'a', 'de', '##tri', '##mental', 'neo', '##p', '##las', '##m', 'affecting', 'p', '##le', '##ural', 'sheets', 'and', 'determining', 'a', 'high', 'rate', 'of', 'mortality', '.'], tokens_id [101, 185, 1513, 12602, 12477, 2646, 15454, 1143, 7301, 10681, 9436, 1918, 113, 182, 9952, 114, 1110, 170, 1260, 19091, 15595, 15242, 1643, 7580, 1306, 12759, 185, 1513, 12602, 8675, 1105, 13170, 170, 1344, 2603, 1104, 14471, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], pos1 is [0, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162

tokens_id is 128, tokens_mask is 128, segment_id is 128, pos is 128, pos2 is 128, pcnn_mask is 128tokens_id is 128, tokens_mask is 128, segment_id is 128, pos is 128, pos2 is 128, pcnn_mask is 128tokens ['here', '##in', 'we', 'character', '##ize', 'vitamin', 'd', 'receptor', '-', 'mediated', 'regulation', 'of', 'k', '##lot', '##ho', 'm', '##rna', 'expression', ',', 'including', 'the', 'identification', 'of', 'vitamin', 'd', 're', '##sp', '##ons', '##ive', 'elements', '(', 'v', '##dre', '##s', ')', 'in', 'the', 'vicinity', 'of', 'both', 'the', 'mouse', 'and', 'human', 'k', '##lot', '##ho', 'genes', '.'], tokens_id [101, 1303, 1394, 1195, 1959, 3708, 23971, 173, 10814, 118, 22060, 8585, 1104, 180, 7841, 5114, 182, 11782, 2838, 117, 1259, 1103, 9117, 1104, 23971, 173, 1231, 20080, 4199, 2109, 3050, 113, 191, 11114, 1116, 114, 1107, 1103, 9748, 1104, 1241, 1103, 10322, 1105, 1769, 180, 7841, 5114, 9077, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

tokens ['using', 'a', 'newly', 'produced', 'anti', '-', 'e', '##por', 'anti', '##body', 'that', 're', '##lia', '##bly', 'detect', '##s', 'the', 'full', '-', 'length', 'is', '##of', '##orm', 'of', 'the', 'e', '##por', 'we', 'show', 'that', 'breast', 'cancer', 'tissue', 'tissue', 'and', 'cells', 'express', 'the', 'e', '##por', 'protein', '.'], tokens_id [101, 1606, 170, 3599, 1666, 2848, 118, 174, 18876, 2848, 14637, 1115, 1231, 4567, 4999, 11552, 1116, 1103, 1554, 118, 2251, 1110, 10008, 24211, 1104, 1103, 174, 18876, 1195, 1437, 1115, 7209, 4182, 7918, 7918, 1105, 3652, 6848, 1103, 174, 18876, 4592, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], pos1 is [0, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 12

['m', 'el', '##ano', '##ma', 'is', 'one', 'of', 'the', 'dead', '##liest', 'c', 'an', '##cer', '##s', ',', 'yet', 'the', 'cells', 'of', 'origin', 'and', 'mechanisms', 'of', 'tumor', 'initiation', 'remain', 'unclear', '.']tokens ['this', 'study', 'aimed', 'at', 'assessing', 'the', 're', '##gene', '##rative', 'effect', 'of', 'p', '-', 't', '##yr', '##oso', '##l', 'in', 'trans', '##ient', 'global', 'cerebral', 'is', '##che', '##mia', 'modeled', 'in', 'adult', 'male', 'w', '##ista', '##r', 'rats', 'by', 're', '##versible', 'o', '##cc', '##lusion', 'of', 'the', 'three', 'major', 'vessels', 'originating', 'from', 'the', 'a', '##ort', '##ic', 'arch', 'and', 'supplying', 'the', 'blood', 'to', 'the', 'brain', '.', 'p', '-', 't', '##yr', '##oso', '##l', 'was', 'administered', 'in', '##tra', '##per', '##ito', '##nea', '##lly', 'in', 'a', 'dose', 'of', '20', 'mg', '/', 'kg', 'over', '10', 'days', 'after', 'surgery', '.'], tokens_id [101, 1142, 2025, 5850, 1120, 23529, 1103, 1231, 27054, 15306, 2629

tokens_id is 128, tokens_mask is 128, segment_id is 128, pos is 128, pos2 is 128, pcnn_mask is 128['interesting', '##ly', ',', 'el', '##a', '-', 'my', '##c', ':', 'par', '##p', '-', '1', '(', '-', '/', '-', ')', 'mice', 'displayed', 'fewer', 'duct', '##al', 't', '##umour', '##s', 'than', 'their', 'el', '##a', '-', 'my', '##c', ':', 'par', '##p', '-', '1', '(', '+', '/', '+', ')', 'counterparts', ',', 'suggesting', 'that', 'par', '##p', '-', '1', 'participates', 'in', 'promoting', 'a', '##cin', '##ar', '-', 'to', '-', 'duct', '##al', 'meta', '##p', '##lasia', ',', 'a', 'key', 'event', 'in', 'pan', '##cre', '##atic', 'cancer', 'initiation', '.']
['mi', '##r', '-', '124', 'expression', 'in', 'breast', 'cancer', 'tissue', 'was', 'measured', 'by', 'quantitative', 'real', '-', 'time', 'p', '##c', '##r', '(', 'q', '##rt', '-', 'p', '##c', '##r', ')', '.']tokens ['purpose', ':', 'disco', '##rda', '##nces', 'between', 'the', 'est', '##rogen', 'receptor', '(', 'er', ')', ',', 'pro', '##ges', '##

tokens ['some', 'of', 'the', 'largest', 'un', '##res', '##ol', '##ved', 'issues', 'are', 'pro', '##state', 'cancer', 'multi', '##fo', '##cal', '##ity', ',', 'limitations', 'of', 'current', 'bio', '##psy', 'strategies', ',', 'sub', '##op', '##ti', '##mal', 'staging', 'by', 'accepted', 'imaging', 'm', '##oda', '##lities', ',', 'less', 'than', 'robust', 'prediction', 'models', 'for', 'in', '##do', '##lent', 'pro', '##state', 'cancer', '##s', ',', 'and', 'safety', 'and', 'efficiency', 'of', 'the', 'established', 'cu', '##rative', 'the', '##rap', '##ies', 'following', 'focal', 'therapy', 'for', 'pro', '##state', 'cancer', '.'], tokens_id [101, 1199, 1104, 1103, 2026, 8362, 4894, 4063, 5790, 2492, 1132, 5250, 19596, 4182, 4321, 14467, 7867, 1785, 117, 13004, 1104, 1954, 25128, 12685, 10700, 117, 4841, 4184, 3121, 7435, 16772, 1118, 3134, 14377, 182, 16848, 16652, 117, 1750, 1190, 17351, 20770, 3584, 1111, 1107, 2572, 13147, 5250, 19596, 4182, 1116, 117, 1105, 3429, 1105, 8096, 1104, 1103, 16

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

